In [1]:
!pip install mne mne-bids


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 9.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import mne
from mne_bids import BIDSPath, read_raw_bids
import re

bids_root = "/content/drive/MyDrive/ieeg_ieds_bids"
bids_path = BIDSPath(
    subject='02',
    task='sleep',
    datatype='ieeg',
    root=bids_root
)

import re
import mne

raw = read_raw_bids(bids_path, verbose=False) #loading raw data

all_chs = raw.ch_names

amyg_prefixes = ["LA", "RA", "LBAA", "RBAA"]           # left/right amygdala
hipp_prefixes = [
    "LH", "RH",
    "LAH", "RAH",
    "LMH", "RMH",
    "LPHG", "RPHG"]   # left/right hippocampus

def is_valid_contact(ch_name, prefixes):  #checks whether the channel name is one of the amygdala/hippocampus prefixes
    if "-" in ch_name:  #exclude all bipolar channels, monopolar contacts 1-4 (signals recorded from one electrode contact)
        return False

    for p in prefixes: #loops through each valid prefix region (LA, RA, LMH, RMH)
        if re.fullmatch(fr"{p}[1-4]$", ch_name):  #uses regular expresstion to match channel names
            return True
    return False

#loops through all channels and keeps only if above function is true AND the prefix matches the labels

amyg_channels = [ch for ch in all_chs if is_valid_contact(ch, amyg_prefixes)]
hipp_channels = [ch for ch in all_chs if is_valid_contact(ch, hipp_prefixes)]

# Combine
selected_channels = amyg_channels + hipp_channels

raw_selected = raw.copy().pick(selected_channels)

print("Amygdala channels:", amyg_channels)
print("Hippocampus channels:", hipp_channels)
print("Final selected channels:", selected_channels)
print("Shape:", raw_selected.get_data().shape)

Amygdala channels: ['RA1', 'RA2', 'RA3']
Hippocampus channels: ['LPHG1', 'LPHG2', 'LPHG3', 'RMH1', 'RMH2', 'RMH3']
Final selected channels: ['RA1', 'RA2', 'RA3', 'LPHG1', 'LPHG2', 'LPHG3', 'RMH1', 'RMH2', 'RMH3']
Shape: (9, 181000)


/tmp/ipython-input-566429452.py:16: RuntimeWarning: MNI152Lin is not an MNE-Python coordinate frame for IEEG data and so will be set to 'unknown'
  raw = read_raw_bids(bids_path, verbose=False) #loading raw data
/tmp/ipython-input-566429452.py:16: RuntimeWarning: DigMontage is only a subset of info. There are 12 channel positions not present in the DigMontage. The channels missing from the montage are:

['LPHG1-LPHG2', 'LPHG2-LPHG3', 'LPHG3-LPHG4', 'RA1-RA2', 'RA2-RA3', 'RA3-RA4', 'REC1-REC2', 'REC2-REC3', 'REC3-REC4', 'RMH1-RMH2', 'RMH2-RMH3', 'RMH3-RMH4'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw = read_raw_bids(bids_path, verbose=False) #loading raw data
/tmp/ipython-input-566429452.py:16: RuntimeWarning: Unable to map the following column(s) to to MNE:
descriptive_SOZ: 
SOZ_laterality: Bila